In [1]:
import cv2
import chassisml
import numpy as np
import getpass
import json
import mxnet as mx
from mxnet import gluon, nd
from mxnet.gluon.model_zoo import vision
import numpy as np

ModuleNotFoundError: No module named 'mxnet'

## Enter credentials
Dockerhub creds and Modzy API Key

In [3]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


In [4]:
model = onnx.load("mobilenetv2-7.onnx")

In [18]:
onnx.checker.check_model(model)

In [20]:
# format sample serialized input into correct format
tensor = onnx.TensorProto()
with open("data/input_0.pb", "rb") as f:
    tensor.ParseFromString(f.read())
array = numpy_helper.to_array(tensor)

In [ ]:
# choose your preferred onnx runtime to run this pytorch model

In [6]:
type(model)

onnx.onnx_ml_pb2.ModelProto

In [13]:
img = cv2.cvtColor(cv2.imread("airplane.jpg"), cv2.COLOR_BGR2RGB)
img.shape

(373, 640, 3)

In [14]:
img_tensor = numpy_helper.from_array(img)

In [16]:
backend.run_model(model, img_tensor)

AttributeError: module 'onnx.backend' has no attribute 'run_model'

In [10]:
model(img)

TypeError: 'ModelProto' object is not callable

## Train model

In [5]:
# Import and normalize data
X_digits, y_digits = datasets.load_digits(return_X_y=True)
X_digits = X_digits / X_digits.max()

n_samples = len(X_digits)

# Split data into training and test sets
X_train = X_digits[: int(0.9 * n_samples)]
y_train = y_digits[: int(0.9 * n_samples)]
X_test = X_digits[int(0.9 * n_samples) :]
y_test = y_digits[int(0.9 * n_samples) :]

# Train Model
logistic = LogisticRegression(max_iter=1000)
print(
    "LogisticRegression mean accuracy score: %f"
    % logistic.fit(X_train, y_train).score(X_test, y_test)
)

# Save small sample input to use for testing later
sample = X_test[:5].tolist()
with open("data/digits_sample.json", 'w') as out:
    json.dump(sample, out)

LogisticRegression mean accuracy score: 0.933333


## Prepare context dict
Initialize anything here that should persist across inference runs

In [6]:
# This will be passed to Chassis:
context = {"model": logistic}

## Write process function

* Must take bytes and context dict as input
* Preprocess bytes, run inference, postprocess model output, return results

In [7]:
def process(input_bytes,context):
    inputs = np.array(json.loads(input_bytes))
    inference_results = context["model"].predict(inputs)
    structured_results = []
    for inference_result in inference_results:
        structured_output = {
            "data": {
                "result": {"classPredictions": [{"class": str(inference_result), "score": str(1)}]}
            }
        }
        structured_results.append(structured_output)
    return structured_results

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [8]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `context` dict containing all variables which should be loaded once and persist across inferences
* Requires `process_fn` defined above

In [9]:
# create Chassis model
chassis_model = chassis_client.create_model(context=context,process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/digits_sample.json'
results = chassis_model.test(sample_filepath)
print(results)

b'[{"data":{"result":{"classPredictions":[{"class":"5","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"2","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"0","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"1","score":"1"}]}}}]'


In [10]:
# test environment and model within Chassis service, must pass filepath here:

# dry run before build
test_env_result = chassis_model.test_env(sample_filepath)
print(test_env_result)

Starting test job... Ok!
{'model_output': 'b\'[{"data":{"result":{"classPredictions":[{"class":"5","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"2","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"0","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"1","score":"1"}]}}}]\'\n'}


## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [11]:
response = chassis_model.publish(
    model_name="Sklearn Logistic Regression Digits Image Classification",
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    modzy_sample_input_path=sample_filepath,
    modzy_api_key=modzy_api_key
)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


In [12]:
if chassis_client.get_job_status(job_id)["result"] is not None:
    print("New model URL: {}".format(chassis_client.get_job_status(job_id)["result"]["container_url"]))
else:
    print("Chassis job failed \n\n {}".format(chassis_client.get_job_status(job_id)))

New model URL: https://integration.modzy.engineering/models/kihikghkee/0.0.1


## Run sample job using Modzy SDK
Submit inference job to our newly-deploy model running on Modzy

In [13]:
from modzy import ApiClient

client = ApiClient(base_url='https://integration.modzy.engineering/api', api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)

[ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "5",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "2",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "8",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "0",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "1",
          "score": "1"
        }
      ]
    }
  }
})]
